## Setup

In [1]:
import os, sys
sys.path.append(os.path.abspath('../../src/'))

import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.optim.lr_scheduler import StepLR, ExponentialLR, LinearLR
from torch.utils.data import DataLoader

from model.spectogram_dataset import SpectrogramDataset

import wandb

wandb.login()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: mldogs (mldogs-wut). Use `wandb login --relogin` to force relogin


True

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


## CNN

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self, dropout_conv_p=0.0, dropout_fc_p=0.0, use_batchnorm=True):
        super(Net, self).__init__()

        # Convolutional and pooling layers
        self.conv1 = nn.Conv2d(1, 16, kernel_size=5)  # Output: (16, 169, 82)
        self.bn1 = nn.BatchNorm2d(16) if use_batchnorm else None  # Batch Norm (optional)
        self.pool1 = nn.MaxPool2d(2, 2)              # Output: (16, 84, 41)

        self.conv2 = nn.Conv2d(16, 32, kernel_size=5) # Output: (32, 80, 37)
        self.bn2 = nn.BatchNorm2d(32) if use_batchnorm else None  # Batch Norm (optional)
        self.pool2 = nn.MaxPool2d(2, 2)              # Output: (32, 40, 18)

        # Dropout after convolutional layers
        self.dropout_conv = nn.Dropout2d(p=dropout_conv_p)  # Dropout2d for 2D data

        # Fully connected layers
        self.fc1 = nn.Linear(32 * 40 * 18, 256)      # Adjusted input size
        self.fc2 = nn.Linear(256, 128)              # Fully connected layer 2
        self.fc3 = nn.Linear(128, 1)              

        # Dropout for fully connected layers
        self.dropout_fc = nn.Dropout(p=dropout_fc_p)

    def forward(self, x):
        # Convolutional layers
        x = self.conv1(x)
        if self.bn1:  # Apply Batch Norm if enabled
            x = self.bn1(x)
        x = F.relu(x)  # Apply ReLU activation
        x = self.pool1(x)  # Pooling
        x = self.dropout_conv(x)  # Dropout (no effect if p=0.0)

        x = self.conv2(x)
        if self.bn2:  # Apply Batch Norm if enabled
            x = self.bn2(x)
        x = F.relu(x)  # Apply ReLU activation
        x = self.pool2(x)  # Pooling
        x = self.dropout_conv(x)  # Dropout (no effect if p=0.0)

        # Flatten
        x = torch.flatten(x, 1)

        # Fully connected layers
        x = self.fc1(x)
        x = F.relu(x)  # Apply ReLU activation
        x = self.dropout_fc(x)  # Dropout (no effect if p=0.0)

        x = self.fc2(x)
        x = F.relu(x)  # Apply ReLU activation
        x = self.dropout_fc(x)  # Dropout (no effect if p=0.0)
        x = torch.sigmoid(self.fc3(x))  # Output layer
        
        # x = torch.sigmoid(self.fc2(x))
        return x


In [4]:
from sklearn.metrics import f1_score

def train(model, criterion, optimizer, scheduler, num_epochs, train_loader, val_loader, model_name = "cnn"):
    model.to(device)
    
    PRINT_STEP = len(train_loader) // 5 - 1
    epochs_without_val_acc_improvement = 0
    best_val_acc = 0.0

    for epoch in range(num_epochs):
        model.train()

        running_loss = 0.0
        correct = 0
        all_labels = []
        all_preds = []

        for  i, data in enumerate(train_loader):
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)
            labels = labels.unsqueeze(1)

            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            preds = (outputs > 0.5).float()  # Binary prediction with threshold 0.5
            all_labels.extend(labels.cpu().numpy().flatten())
            all_preds.extend(preds.detach().cpu().numpy().flatten())

            correct += (preds == labels).float().mean().item()
            
            running_loss += loss.item()

            if i % PRINT_STEP == PRINT_STEP-1:
                accuracy = correct / PRINT_STEP
                loss = running_loss / PRINT_STEP
                step = epoch * len(train_loader) + i
                wandb.log({
                        "train/accuracy": accuracy,
                        "train/loss": loss
                    },
                    step=step
                )
                running_loss = 0.0
                correct = 0

        f1 = f1_score(all_labels, all_preds, average='macro')
        print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {running_loss / len(train_loader):.4f}, F1 Score: {f1:.4f}")
        
        if scheduler is not None:
            scheduler.step()

        # Validation
        model.eval()

        val_loss = 0.0
        val_correct = 0

        with torch.no_grad():
            for j, data in enumerate(val_loader):
                inputs, labels = data
                inputs = inputs.to(device)
                labels = labels.to(device)
                labels = labels.unsqueeze(1)

                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()

                # Binary predictions
                preds = (outputs > 0.5).float()
                val_correct += (preds == labels).float().mean().item()

        accuracy = val_correct / len(val_loader)
        loss = val_loss / len(val_loader)
        wandb.log({
                "validation/accuracy": accuracy,
                "validation/loss": loss
            },
            step=(epoch + 1) * len(train_loader)
        )

        if accuracy > best_val_acc:
            best_val_acc = accuracy
            epochs_without_val_acc_improvement = 0
        else:
            epochs_without_val_acc_improvement += 1
        if epochs_without_val_acc_improvement >= 10:
            print("10 epochs without a val accuracy improvement. Stopping the train")
            break

    torch.save(model.state_dict(), f"{model_name}_dict.pth")
    torch.save(model, f"{model_name}.pth")
    print("Training complete.")
    
def test(model, test_loader, criterion):
    model.eval()

    test_loss = 0.0
    test_correct = 0
    all_labels = []
    all_preds = []

    with torch.no_grad():
        for data in test_loader:
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)
            labels = labels.unsqueeze(1)

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            test_loss += loss.item()

            # Binary predictions
            preds = (outputs > 0.5).float()
            all_labels.extend(labels.cpu().numpy().flatten())
            all_preds.extend(preds.detach().cpu().numpy().flatten())

            test_correct += (preds == labels).float().mean().item()

    accuracy = test_correct / len(test_loader)
    loss = test_loss / len(test_loader)
    f1 = f1_score(all_labels, all_preds, average='macro')

    print(f'Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}, Test F1 Score: {f1:.4f}')
    wandb.log({
            "test/accuracy": accuracy,
            "test/loss": loss,
            "test/f1_score": f1
        }
    )

## Data

In [5]:
from utils.dataset_creator import SpecgramsRandomFilter, SpecgramsSilentFilter

DATASETS_PARENT_PATH = 'datasets'
DATASET_PATH = 'dataset'
DATA_DIR = os.path.join('../../', DATASETS_PARENT_PATH, DATASET_PATH)

In [4]:
from prepare_datasets import create_datasets

create_datasets(DATASETS_PARENT_PATH, DATASET_PATH, [
    SpecgramsSilentFilter(k=1.1)
])

DatasetCreator: Exporting the dataset with the following parameters:
    n_fft=1024
    n_mels=86
    duration_seconds=2s
    multithreading=True
Class 0 recordings count: 270
Class 1 recordings count: 630
Image properties:
    width=173px
    height=86px
Removing directory /home/migoox/dev/ml-dogs-sharepoint/src/../datasets/dataset/train/0
Removing directory /home/migoox/dev/ml-dogs-sharepoint/src/../datasets/dataset/train/1
Finished [900/900]Dataset has been exported. Elapsed time: 131.48113918304443s.
DatasetCreator: Exporting the dataset with the following parameters:
    n_fft=1024
    n_mels=86
    duration_seconds=2s
    multithreading=True
Class 0 recordings count: 90
Class 1 recordings count: 210
Image properties:
    width=173px
    height=86px
Removing directory /home/migoox/dev/ml-dogs-sharepoint/src/../datasets/dataset/validation/0
Removing directory /home/migoox/dev/ml-dogs-sharepoint/src/../datasets/dataset/validation/1
Finished [300/300]Dataset has been exported. Elapse

In [8]:
INPUT_WIDTH = 173
INPUT_HEIGHT = 86

LR_DECAY = 0.95
EPOCHS = 40
LEARNING_RATE = 0.001
BATCH_SIZE = 32

DROPOUT_CONV = 0.3
DROPOUT_FC = 0.3

USE_BATCHNORM = False

## Training

In [10]:
import itertools


name = f"network"

wandb.init(
    name=name,
    project="iml_final",
    config={
        "learning_rate": LEARNING_RATE,
        "learning_rate_decay": LR_DECAY,
        "batch_size": BATCH_SIZE,
        "dropout_conv_p": DROPOUT_CONV,
        "dropout_fc_p": DROPOUT_FC,
        "use_batchnorm": USE_BATCHNORM,
        "input_resolution": (INPUT_WIDTH, INPUT_HEIGHT),
        "scheduler": "none",
        "architecture": "CNN",
        "dataset": "DAPS"
    }
)

model = Net(dropout_conv_p=DROPOUT_CONV,
            dropout_fc_p=DROPOUT_FC,
            use_batchnorm=USE_BATCHNORM)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
schedulers = [None,
                StepLR(optimizer, step_size=10, gamma=LR_DECAY),
                ExponentialLR(optimizer, gamma=LR_DECAY),
                LinearLR(optimizer, start_factor=0.333, end_factor=1.0, total_iters=EPOCHS)]

transform = transforms.ToTensor()

train_dataset = SpectrogramDataset(data_dir=os.path.join(DATA_DIR, "train"), transform=transform)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

val_dataset = SpectrogramDataset(data_dir=os.path.join(DATA_DIR, "validation"), transform=transform)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

test_dataset = SpectrogramDataset(data_dir=os.path.join(DATA_DIR, "test"), transform=transform)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True)

# Train and evaluate
train(model, criterion, optimizer, schedulers[3], EPOCHS, train_loader, val_loader, model_name=name)
test(model, test_loader, criterion)

wandb.finish()

Epoch 1/40, Loss: 0.0220, F1 Score: 0.4006
Epoch 2/40, Loss: 0.0178, F1 Score: 0.4753
Epoch 3/40, Loss: 0.0110, F1 Score: 0.7703
Epoch 4/40, Loss: 0.0083, F1 Score: 0.8600
Epoch 5/40, Loss: 0.0061, F1 Score: 0.9097
Epoch 6/40, Loss: 0.0026, F1 Score: 0.9347
Epoch 7/40, Loss: 0.0022, F1 Score: 0.9512
Epoch 8/40, Loss: 0.0025, F1 Score: 0.9670
Epoch 9/40, Loss: 0.0014, F1 Score: 0.9760
Epoch 10/40, Loss: 0.0027, F1 Score: 0.9749
Epoch 11/40, Loss: 0.0009, F1 Score: 0.9817
Epoch 12/40, Loss: 0.0005, F1 Score: 0.9879
Epoch 13/40, Loss: 0.0008, F1 Score: 0.9872
Epoch 14/40, Loss: 0.0012, F1 Score: 0.9910
Epoch 15/40, Loss: 0.0007, F1 Score: 0.9910
Epoch 16/40, Loss: 0.0018, F1 Score: 0.9936
Epoch 17/40, Loss: 0.0007, F1 Score: 0.9961
Epoch 18/40, Loss: 0.0006, F1 Score: 0.9922
Epoch 19/40, Loss: 0.0014, F1 Score: 0.9951
Epoch 20/40, Loss: 0.0003, F1 Score: 0.9940
Epoch 21/40, Loss: 0.0005, F1 Score: 0.9953
Epoch 22/40, Loss: 0.0004, F1 Score: 0.9951
Epoch 23/40, Loss: 0.0011, F1 Score: 0.99

test/accuracy,▁
test/f1_score,▁
test/loss,▁
train/accuracy,▁▁▃▄▄▅▅▆▇▇▇▇████████████████████████████
train/loss,███▇▆▄▃▂▂▂▂▂▂▂▂▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
validation/accuracy,▁▂▅▆▆▆▇▇▇▇▇▇████████▇█████████████
validation/loss,█▇▃▂▂▂▂▃▂▁▃▃▄▁▂▂▄▂▃▃▅▄▁▂▇▃▃▄▄▅▅▂▆▄
test/accuracy,0.94337
test/f1_score,0.93818
test/loss,0.24637
train/accuracy,0.99821
